In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [14]:
df = pd.read_csv('data8.csv')
data = df
#data['TreatmentA'] = np.where(df['Treatment'] == 'A', 1, 0)
#data['TreatmentB'] = np.where(df['Treatment'] == 'B', 1, 0)
#data['TreatmentC'] = np.where(df['Treatment'] == 'C', 1, 0)
data.replace(['A','B','C'], [1,2,3], inplace = True)
data.replace(['I','G','E'], [0,1,2], inplace = True)
data

,Age,Treatment,Outcome
0,23,3,0
1,19,3,0
2,27,3,0
3,29,3,0
4,30,2,0
5,23,2,0
6,43,2,0
7,33,2,0
8,37,3,0
9,29,2,0


In [3]:
model = GaussianNB()
#model.fit(data[['Age','TreatmentA','TreatmentB','TreatmentA']], data['Outcome'])
model.fit(data[['Age','Treatment']], data['Outcome'])


GaussianNB()

In [4]:
model2 = MultinomialNB()
#model2.fit(data[['Age','TreatmentA','TreatmentB','TreatmentC']], data['Outcome'])
model2.fit(data[['Age','Treatment']], data['Outcome'])


MultinomialNB()

In [10]:
#!pip install git+https://github.com/remykarem/mixed-naive-bayes#egg=mixed_naive_bayes

  Cloning https://github.com/remykarem/mixed-naive-bayes to /private/var/folders/zr/nth0p58d0lncg_nv89hbpzsm0000gn/T/pip-install-h3zb9i1f/mixed-naive-bayes_80d0d6ed364549c4a87ca908c974692d
  Running command git clone -q https://github.com/remykarem/mixed-naive-bayes /private/var/folders/zr/nth0p58d0lncg_nv89hbpzsm0000gn/T/pip-install-h3zb9i1f/mixed-naive-bayes_80d0d6ed364549c4a87ca908c974692d
  Resolved https://github.com/remykarem/mixed-naive-bayes to commit f1c020e79d156efec57c79e0c651c809af3a8722
  Created wheel for mixed-naive-bayes: filename=mixed_naive_bayes-0.0.2-py3-none-any.whl size=11236 sha256=fa75bed5a0c1a45b8f7edb9f4b67dc190dec9cd9a521f9df04973dd2fa033cac
  Stored in directory: /private/var/folders/zr/nth0p58d0lncg_nv89hbpzsm0000gn/T/pip-ephem-wheel-cache-joxitsmv/wheels/eb/77/fe/f09fe070564c9415112738dbebfae8b6afc5b6b0bd143797f5
Successfully built mixed-naive-bayes
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [17]:
from mixed_naive_bayes import MixedNB

model3 = MixedNB()
#model3.fit(data[['Age','TreatmentA','TreatmentB','TreatmentA']], data['Outcome'])
model3.fit(data[['Age','Treatment']], data['Outcome'])


MixedNB(alpha=0.5, var_smoothing=1e-09)

In [5]:
test = pd.DataFrame(
    np.array([[50, 1],
              [50, 2],
              [50, 3]]
             ),
    columns=['Age', 'Treatment'])

test

,Age,Treatment
0,50,1
1,50,2
2,50,3


In [6]:
result = model.predict_proba(test)
result

array([[0.23612869, 0.76208971, 0.0017816 ],
       [0.2211288 , 0.71560852, 0.06326268],
       [0.32634237, 0.49369537, 0.17996225]])

In [7]:
result2 = model2.predict_proba(test)
result2

array([[0.27223028, 0.61387899, 0.11389073],
       [0.22558717, 0.55884113, 0.2155717 ],
       [0.16937092, 0.46093579, 0.36969329]])

In [18]:
result3 = model3.predict_proba(test)
result3

array([[0.00178132, 0.76197053, 0.23609176],
       [0.06325624, 0.71553573, 0.22110631],
       [0.17991019, 0.49355255, 0.32624796]])

In [22]:
resultF = pd.DataFrame(columns=['Ineffective','Good','Excellent'])
for i in range(1, 4):
    one = model.predict_proba([[50]])
    two = model2.predict_proba([[i]])
    y = np.array(data['Outcome'])
    yNum = [len(y[y == 0]), len(y[y == 1]), len(y[y == 2])]
    P = (one[0] * two[0])/yNum
    n = P/sum(P)
    resultF.loc[resultF.columns[i-1]] = n

resultF['Treatment'] = ['A', 'B', 'C']
resultF = resultF.set_index('Treatment')
resultF

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)

In [20]:
# Question A
table = pd.DataFrame(result3, columns=['Excellent','Good','Ineffective'], index=['A','B','C'])
table = table[['Ineffective','Good','Excellent']]
table

,Ineffective,Good,Excellent
A,0.236092,0.761971,0.001781
B,0.221106,0.715536,0.063256
C,0.326248,0.493553,0.179910


In [9]:
# Question B
# I would recommend a 50-year old patient to use treatment A because it has the highest probability 
# turn out to be good and lowest probability of being ineffective, which means that treatment A has the 
# highest probability of being good or excellent. For treatment C, even it has the highest probability 
# of being excellent but it also has the highest chance of being ineffective, which would be too risky.